In [1]:
import h5py
import numpy as np

In [2]:
# Vacuum wavelength [um]
wl_0= 0.8337  

#Calcite Uniaxial, negative
m_p_x= 1.658+0j
m_p_y= 1.658+0j
m_p_z= 1.486+0j
m_p_xyz= [m_p_x, m_p_y, m_p_z]

# list of refractive index of medium 
m_m_list= [1.00, 1.329]
#m_m_list= [1.00]

r_v_base_list= [0.3,0.4] # [um] list of r_v_base grid points
bc_ratio_list= [1] # list of bc ratio grid points
ab_ratio_list= [1] # list of ab ratio grid points
gre_beta_list= [0]  # list of beta grid points

num_orientations= 400  # sets of euler angles (alpha,beta,gamma) 

filename= "pcas_ocbs_simulated_data.hdf5"

with h5py.File(filename, "w") as f:
    grp_target= f.create_group("target")
    grp_target.attrs['light_source']= 'Thorlabs SLD830S-A10 operated at I=140mA, T=25C, center wavelength was 0.8337um.'
    grp_target.attrs['polarization state']= 'left-handed circulat polarization with Jones vector: E0_inc_x(theta)= 1/np.sqrt(2), E0_inc_y(phi)= 1j/np.sqrt(2)'
    grp_target.attrs['wl_0_um']= wl_0
    grp_target.attrs['num_orientations']= num_orientations
 
    dset_m_p_xyz= grp_target.create_dataset("m_p_xyz", data= m_p_xyz, dtype=np.complex128)
    dset_m_m= grp_target.create_dataset("m_m_list", data= m_m_list, dtype=np.float64)
    dset_r_v_base= grp_target.create_dataset("r_v_base_list", data= r_v_base_list, dtype=np.float64)
    dset_bc_ratio= grp_target.create_dataset("bc_ratio_list", data= bc_ratio_list, dtype=np.float64)
    dset_ab_ratio= grp_target.create_dataset("ab_ratio_list", data= ab_ratio_list, dtype=np.float64)
    dset_gre_beta= grp_target.create_dataset("gre_beta_list", data= gre_beta_list, dtype=np.float64)
    dset_euler_angles= grp_target.create_dataset("euler_angles", data= np.zeros((num_orientations,3),dtype=np.float64))
    dset_euler_angles.attrs['definition']= 'Euler angles (alpha, beta, gamma) of rotation of the particle frame from the laboratory frame, where 0 <= alpha < 2pi, 0 <= beta < pi, 0 <= gamma < 2pi'

    grp_simulated_data= grp_target.create_group("simulated_data")
    grp_simulated_data.attrs['scattering calculations']='discrete dipole approximation: block-DDA_Py'
    grp_simulated_data.attrs['particle orientation']='ramdomly chosen sets of Euler angles (alpha, beta, gamma) where 0 <= alpha < 2pi, 0 <= beta < pi, 0 <= gamma < 2pi'
    grp_simulated_data.attrs['units']='r_ve:[um], euler_angles:[radian], C_abs and Cext:[um2], S:[um]'
    grp_simulated_data.attrs['definition of the complex amplitude S']= 'S(0)_theta:= S11(0)+1j*S12(0), S(0)_phi:= S22(0)-1j*S21(0), S(180):= (S11(180)+S22(180)+1j*S12(180)-1j*S21(180))/np.sqrt(2), where S11, S12, S21, S22 is the element of amplitude scattering matrix defined in Mischenko textbook 2000'

    dset_r_ve= grp_simulated_data.create_dataset("r_ve",(len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list)), dtype=np.float64)
    dset_r_ve.attrs['definition']= 'volume equivalent radius [um] calculated from the discretized particle volume'
    
    dset_C_abs= grp_simulated_data.create_dataset("C_abs",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list), num_orientations), dtype=np.float64)
    dset_C_abs.attrs['definition']= 'absorption cross section [um2]'
    dset_C_ext= grp_simulated_data.create_dataset("C_ext",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list), num_orientations), dtype=np.float64)
    dset_C_ext.attrs['definition']= 'extinction cross section [um2]'
    dset_S_fw_PCAS_theta= grp_simulated_data.create_dataset("S_fw_PCAS_theta",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list), num_orientations), dtype=np.complex128)
    dset_S_fw_PCAS_theta.attrs['definition']= 'S(0)_theta [um], the forward-scattering amplitudes of s-pol(along theta-vector) channel'
    dset_S_fw_PCAS_phi= grp_simulated_data.create_dataset("S_fw_PCAS_phi",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list), num_orientations), dtype=np.complex128)
    dset_S_fw_PCAS_phi.attrs['definition']= 'S(0)_phi [um], the forward-scattering amplitudes of p-pol(along phi-vector) channel'
    dset_S_bk_OCBS= grp_simulated_data.create_dataset("S_bk_OCBS",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list), num_orientations), dtype=np.complex128)
    dset_S_bk_OCBS.attrs['definition']= 'the complex backward-scattering amplitude S(180) [um]'

    dset_C_abs_mie= grp_simulated_data.create_dataset("C_abs_mie",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list)), dtype=np.float64)
    dset_C_abs_mie.attrs['definition']= 'absorption cross section [um2] of the volume-equivalent sphere of the target shape'
    dset_C_ext_mie= grp_simulated_data.create_dataset("C_ext_mie",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list)), dtype=np.float64)
    dset_C_ext_mie.attrs['definition']= 'extinction cross section [um2] of the volume-equivalent sphere of the target shape'
    dset_S_fw_PCAS_mie= grp_simulated_data.create_dataset("S_fw_PCAS_mie",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list)), dtype=np.complex128)
    dset_S_fw_PCAS_mie.attrs['definition']= 'complex forward-scattering amplitude S(0) of the volume-equivalent sphere of the target shape'
    dset_S_bk_OCBS_mie= grp_simulated_data.create_dataset("S_bk_OCBS_mie",(len(m_m_list), len(r_v_base_list), len(bc_ratio_list), len(ab_ratio_list), len(gre_beta_list)), dtype=np.complex128)
    dset_S_bk_OCBS_mie.attrs['definition']= 'the complex backward-scattering amplitude S(180) of the volume-equivalent sphere of the target shape'
    

In [3]:
f= h5py.File(filename, "a")

In [4]:
f['target'].keys()

<KeysViewHDF5 ['ab_ratio_list', 'bc_ratio_list', 'euler_angles', 'gre_beta_list', 'm_m_list', 'm_p_xyz', 'r_v_base_list', 'simulated_data']>

In [5]:
f['target']['simulated_data']['r_ve'][0]

array([[[0.]]])

In [6]:
f['target']['m_p_xyz'][:]

array([1.658+0.j, 1.658+0.j, 1.486+0.j])

In [7]:
f['target']['simulated_data']['r_ve'][:,:,:,:]=0

In [8]:
f['target'].attrs['wl_0_um']

np.float64(0.8337)

In [9]:
f.close()